Considerando el pagelink_sample.csv, usando una representación de grafos realizar una función genérica que nos permita calcular los contenidos que se encuentran a un grado de separación de cualquier identificador de contenido de la wikipedia. Mostrar el funcionamiento de la implementación con algún contenido incluido en el set de datos (⭐⭐⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=7422fb3ab6b3c46f9b15e48bcbed0318445bb96951011471ab3402b08f9dfbf4
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launch

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_page = sqlContext.read.csv('/content/drive/MyDrive/pagelinks_sample.csv', header=True, inferSchema=True, multiLine=True, escape='"')
rdd_page= df_page.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
sqlContext2 = SQLContext(sc)
df_contents = sqlContext2.read.csv('/content/drive/MyDrive/contents.csv', header=True, inferSchema=True, multiLine=True, escape='"')
rdd_contents= df_contents.rdd

Uso otro formato para los dos rdds

In [ ]:
rdd_page_filtrado = rdd_page.map(lambda x: (x.pl_title, x.pl_from))

In [ ]:
rdd_contents_filtrado = rdd_contents.map(lambda x: (x.title, x.id))

Necesito trabajar con los artiuclos cuyo titulo figura en contens_csv, ya que para armar el grafo necesito trabajar con las ids asociadas de cada artiuclo. Para esto hago join

In [ ]:
rdd_articulos_ids = rdd_page_filtrado.join(rdd_contents_filtrado).cache()

Agrupo los datos por ids, en donde forma una lista de adyacencia la cual representa a un grafo.

In [ ]:
rdd_articulos_ids = rdd_articulos_ids.map(lambda x: (x[1][0], [x[1][1]]))

In [ ]:
rdd_agrupados_id = rdd_articulos_ids.reduceByKey(lambda x, y: x + y).cache()

Una vez armado el grafo, defino la funcion que me va a retornar una lista de todos los nodos adyacentes que son adyacentes a la clave.

In [ ]:
def devolver_lista_grado_1_separacion(clave):

  #Creo una lista de listas (un elemento) con los nodos adyacentes del nodo clave.
  rdd_nodos_adyacentes = rdd_agrupados_id.filter(lambda x: x[0] == clave).map(lambda x: x[1])
  #Creo la lista de nodos adyacente a la clave en un formato trabajable.
  lista_adyacentes = rdd_nodos_adyacentes.take(1)[0]

  #Creo un rdd en donde estan los nodos adyacentes de los nodos adyacentes a la clave, luego lo llevo a una lista para poder trabajarlo.
  rdd_adyacentes_de_adyacentes = rdd_agrupados_id.filter(lambda x: x[0] in lista_adyacentes).map(lambda x: (1, x[1]))
  #Agrupo todas las listas de los nodos de los adyacentes de los adyacentes de la clave.
  rdd_adyacentes_de_adyacentes = rdd_adyacentes_de_adyacentes.reduceByKey(lambda x, y: x + y).map(lambda x: x[1]).cache()
  rdd_adyacentes_de_adyacentes = sc.parallelize(rdd_adyacentes_de_adyacentes.take(1)[0], 1)
  #Elimino nodos repetidos y creo una lista de los adyacentes de los adyacentes del nodo clave.
  Lista_adyacentes_de_adyacentes = rdd_adyacentes_de_adyacentes.distinct().collect()
  #Si esta en la lista de adyacentes de adyacente lo remuevo.
  #Creo la Lista que voy a devolver, ya que necesito fijarme los nodos que no son adyacentes al nodo clave y que no sean
  #la clave (Puede estar en lista_adyacentes_adyacntes ya que no es la version final de la misma)
  Lista_devolver = rdd_agrupados_id.filter(lambda x: (x[0] in Lista_adyacentes_de_adyacentes) and (clave not in x[1]) and (x[0] not in lista_adyacentes) and (clave != x[0])).map(lambda x: x[0]).collect()
  Lista_auxiliar = []

  #Agrego aquellos nodos que no aparecen en la lista de rdd_agrupados_id ya que estos nodos no tiene adyacentes, probocando
  #que sean de grado 1 con respecto al nodo clave, siempre y cuando no sean adyacentes a la clave o sea la clave.
  for i in Lista_adyacentes_de_adyacentes:
    if((i not in lista_adyacentes) and (i not in Lista_devolver) and (i != clave)):
        Lista_auxiliar.append(i)

  Lista_devolver.extend(Lista_auxiliar)
  return Lista_devolver



Funcionamiento del codigo

In [ ]:
Lista_grado_1_separacion_ej1 = devolver_lista_grado_1_separacion(2389124)

In [ ]:
Lista_grado_1_separacion_ej2 = devolver_lista_grado_1_separacion(300)

In [ ]:
Lista_grado_1_separacion_ej3 = devolver_lista_grado_1_separacion(434367)

Aclaracion: La clave elegida tiene que tener adyacentes para que funcione el algoritmo, sino no va a estar represebtado en la lista de adyacencia. Elegi esta implentacion del grafo porque los nodos que no tienen adyacentes no me sirven para realizar el ejercicio porque necesito solo los que tienen un grado de separacion y van a estar ocupando un espacio innecesario.